In [ ]:
import pandas as pd
#import pylab
#import scipy
import matplotlib.pyplot as plt
import matplotlib #so I can call next line
matplotlib.style.use('ggplot')
import seaborn as sns
import numpy as np

#visuals
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format #7,123,001.34
#'{:20,.2f}'.format #change pandas display format
pd.options.display.max_rows = 20 
#pd.get_option("display.max_rows")


from __future__ import division #so I can have float as std and int as //

#my own functions
#from supportFunctions import * #you CAN'T RELOAD functions

# Processing GNSS measurements

This presentation aims to explain how to calculate pseudo ranges from the obtainable from the Android API 24+.

This code is the equivalent of `ProcessGnssMeas.m` Matlab code from [Google GPS Measurement Tools](https://github.com/google/gps-measurement-tools).

## short intro to GNSS ranges

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Bad_gdop.png", width=50%,height=60>


Pseudorange is transmission time from satellite to the receiver, calculated as $$L = \frac{T_{receiver \atop arrival}
-T^{satellite \atop transmission}}{c}$$. 
It is called pseudorange as <https://www.youtube.com/playlist?list=PL635065C128E0DB27>


# Loading and reading data

In [ ]:
data_file = "../GNSSdatasets/raw.csv"
#data_file = "./workshop_trials01.txt"
print("reading %s" % data_file)
df_GNSS = pd.read_csv(data_file, delimiter = ",", error_bad_lines=False,header=0,\
                     encoding = 'utf-8-sig',na_values = ["NULL",""],engine ='c');
                     #,nrows = 1000); #parse_dates = ['date'],index_col=0,header = 0,skiprows=range(0, 11),
df_GNSS.head()

In [ ]:
df_GNSS.dtypes

In [ ]:
totalWeekSecs = 7*24*3600
totalWeekSecs #in integer so calculations are done properly

My values are diff from [Frank sample](https://docs.google.com/spreadsheets/d/1e9JaVn9hq04tEVoCDkO7EHi8LdDyf0X7_IuQZ3U1b-E/edit#gid=80118229), is this due to accuracy? We need to check this vs Matlab code.


In [ ]:
GPSWeek = (-df_GNSS.FullBiasNanos*1e-9/totalWeekSecs).astype('int')
tRx_ns = df_GNSS.TimeNanos+df_GNSS.TimeOffsetNanos - (df_GNSS.FullBiasNanos+df_GNSS.BiasNanos)-(GPSWeek*totalWeekSecs*1e9)
PR_ms = (tRx_ns-df_GNSS.ReceivedSvTimeNanos)*1e-6
PR_m = PR_ms*299792458*1e-3
PR_m.head()

## Problems with accuracy of calculations

Excel, the same as python has problem with large numbers, if you use floats. Using int removes the problem

In [ ]:
x=-1151285108458178048
y=x-1
z=x-1+1
print 'x={:}\ny={:}\nz={:}'.format(x,y,z)
print x-y
print z-x